In [180]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import time
import numpy as np

In [151]:
def run_notebook_extract_mlp_anfis(notebook_path, timeout=600, kernel_name="python3"):
    """
    Execute a Jupyter Notebook and extract specified variables.

    Args:
        notebook_path (str): Path to the notebook file.
        timeout (int): Timeout for notebook execution.
        kernel_name (str): Kernel name for execution.

    Returns:
        dict: A dictionary of the extracted variables.
    """
    # Load the notebook
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)

    # Create an ExecutePreprocessor instance
    ep = ExecutePreprocessor(timeout=timeout, kernel_name=kernel_name)

    # Execute the notebook
    try:
        ep.preprocess(notebook, {'metadata': {'path': '.'}})
    except Exception as e:
        raise RuntimeError(f"Error executing notebook {notebook_path}: {e}")

    # Extract variables from the notebook's last cell outputs
    extracted_vars = {}
    for cell in notebook.cells:
        if cell.cell_type == "code" and "outputs" in cell:
            for output in cell.outputs:
                if output.output_type == "stream" and output.name == "stdout":
                    if "Accuracy: " in output.get("text", ""):
                        print(output.get("text", ""))  # Print the matching output
                        extracted_vars = output.get("text", "")
                    
    return extracted_vars

In [152]:
def run_notebook_extract_variables_results(notebook_path, timeout=600, kernel_name="python3"):
    """
    Execute a Jupyter Notebook and extract specified variables.

    Args:
        notebook_path (str): Path to the notebook file.
        timeout (int): Timeout for notebook execution.
        kernel_name (str): Kernel name for execution.

    Returns:
        dict: A dictionary of the extracted variables.
    """
    # Load the notebook
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)

    # Create an ExecutePreprocessor instance
    ep = ExecutePreprocessor(timeout=timeout, kernel_name=kernel_name)

    # Execute the notebook
    try:
        ep.preprocess(notebook, {'metadata': {'path': '.'}})
    except Exception as e:
        raise RuntimeError(f"Error executing notebook {notebook_path}: {e}")

    # Extract variables from the notebook's last cell outputs
    extracted_vars = {}
    for cell in notebook.cells:
        if cell.cell_type == "code" and "outputs" in cell:
            for output in cell.outputs:
                if output.output_type == "stream" and output.name == "stdout":
                    if 'MLP R-Value: ' in output.get("text", ""):
                        print(output.get("text", ""))  # Print the matching output
                        mlp_rvalue = output.get("text", "")
                    elif 'Anfis R-Value: ' in output.get("text", ""):
                        print(output.get("text", ""))
                        anfis_rvalue = output.get("text", "")
                    elif 'Average MLP portfolio appreciation: ' in output.get("text", ""):
                        print(output.get("text", ""))
                        general_result = output.get("text", "")
                    
    return mlp_rvalue, anfis_rvalue, general_result

In [153]:
# Process all scripts:

mlp_path = "../Multilayer Perceptron/multi_layer_perceptron.ipynb"
anfis_path = "../Adaptative Neuro-Fuzzy Inference System/anfis.ipynb"
results_path = "results.ipynb"

mlp_results = []
anfis_results = []
mlp_rvalues = []
anfis_rvalues = []
general_results = []

for i in range(100):
    print(f'Experiment {i+1}:')
    
    # Run MLP Script
    try:
        print(f'processing {mlp_path}')
        results = run_notebook_extract_mlp_anfis(mlp_path)
        mlp_results.append(results)
    except Exception as e:
        print(f"Error processing {mlp_path}: {e}")

    # Run ANFIS Script
    success = False
    while not success:
        try:
            print(f'processing {anfis_path}')
            results = run_notebook_extract_mlp_anfis(anfis_path)
            anfis_results.append(results)
            success = True
        except Exception as e:
            print(f"Error processing {anfis_path}: {e}")

    
    # Run Results Script
    
    try:
        print(f'processing {results_path}')
        mlp_rvalue, anfis_rvalue, general_result = run_notebook_extract_variables_results(results_path)
        mlp_rvalues.append(mlp_rvalue)
        anfis_rvalues.append(anfis_rvalue)
        general_results.append(general_result)
    except Exception as e:
        print(f"Error processing {results_path}: {e}")

Experiment 1:
processing ../Multilayer Perceptron/multi_layer_perceptron.ipynb
Accuracy: 0.684931506849315
Precision: 0.04035087719298246
Recall (Sensitivity): 0.23711340206185566
F1-Score: 0.06896551724137931

processing ../Adaptative Neuro-Fuzzy Inference System/anfis.ipynb
Accuracy: 0.5728056823947235
Precision: 0.07035755478662054
Recall (Sensitivity): 0.6288659793814433
F1-Score: 0.12655601659751037

processing results.ipynb
MLP R-Value: 0.02496640381353648

Anfis R-Value: 0.16122608488562484

Average MLP portfolio appreciation: 4.960%
Average ANFIS portfolio appreciation: 9.408%
Average Market ETF appreciation: 8.565%

Experiment 2:
processing ../Multilayer Perceptron/multi_layer_perceptron.ipynb
Accuracy: 0.7371892440385591
Precision: 0.07128309572301425
Recall (Sensitivity): 0.36082474226804123
F1-Score: 0.11904761904761904

processing ../Adaptative Neuro-Fuzzy Inference System/anfis.ipynb
Error processing ../Adaptative Neuro-Fuzzy Inference System/anfis.ipynb: Error executing 

In [165]:
# Process unstructured data: MLP Results
mlp_results_dict = {}

for entry in mlp_results:
    for line in entry.splitlines():
        key, value = line.split(": ")
        value = float(value)  # Convert string to float
        if key not in mlp_results_dict:
            mlp_results_dict[key] = []
        mlp_results_dict[key].append(value)

In [167]:
# Process unstructured data: ANFIS Results
anfis_results_dict = {}

for entry in anfis_results:
    for line in entry.splitlines():
        key, value = line.split(": ")
        value = float(value)  # Convert string to float
        if key not in anfis_results_dict:
            anfis_results_dict[key] = []
        anfis_results_dict[key].append(value)

In [173]:
# Process unstructured data: MLP R-values
mlp_rvalues_dict = {}

for entry in mlp_rvalues:
    for line in entry.splitlines():
        key, value = line.split(": ")
        value = float(value)  # Convert string to float
        if key not in mlp_rvalues_dict:
            mlp_rvalues_dict[key] = []
        mlp_rvalues_dict[key].append(value)

In [175]:
# Process unstructured data: ANFIS R-values
anfis_rvalues_dict = {}

for entry in anfis_rvalues:
    for line in entry.splitlines():
        key, value = line.split(": ")
        value = float(value)  # Convert string to float
        if key not in anfis_rvalues_dict:
            anfis_rvalues_dict[key] = []
        anfis_rvalues_dict[key].append(value)

In [195]:
# Process unstructured data: General Results
general_results_dict = {}

for entry in general_results:
    for line in entry.splitlines():
        key, value = line.split(": ")
        value = float(value.strip('%'))  # Convert string to float
        if key not in general_results_dict:
            general_results_dict[key] = []
        general_results_dict[key].append(value)

In [211]:
# Final Results (averages):

print(72*'-')
print('MLP measures:')
print(f'Accuracy: {np.average(mlp_results_dict["Accuracy"])}')
print(f'Precision: {np.average(mlp_results_dict["Precision"])}')
print(f'Recall (Sensitivity): {np.average(mlp_results_dict["Recall (Sensitivity)"])}')
print(f'F1-Score: {np.average(mlp_results_dict["F1-Score"])}')
print(72*'-')
print('ANFIS measures:')
print(f'Accuracy: {np.average(anfis_results_dict["Accuracy"])}')
print(f'Precision: {np.average(anfis_results_dict["Precision"])}')
print(f'Recall (Sensitivity): {np.average(anfis_results_dict["Recall (Sensitivity)"])}')
print(f'F1-Score: {np.average(anfis_results_dict["F1-Score"])}')
print(72*'-')
print('General results:')
print(f'MLP R-value: {np.average(mlp_rvalues_dict["MLP R-Value"])}')
print(f'ANFIS R-value: {np.average(anfis_rvalues_dict["Anfis R-Value"])}')
print(f'Average MLP portfolio appreciation: {np.average(general_results_dict["Average MLP portfolio appreciation"])}')
print(f'Average ANFIS portfolio appreciation: {np.average(general_results_dict["Average ANFIS portfolio appreciation"])}')
print(f'Average Market portfolio appreciation: {general_results_dict["Average Market ETF appreciation"][0]}')

------------------------------------------------------------------------
MLP measures:
Accuracy: 0.6787113140537798
Precision: 0.06656410026928727
Recall (Sensitivity): 0.42350515463917526
F1-Score: 0.11484480825536726
------------------------------------------------------------------------
ANFIS measures:
Accuracy: 0.5760527650938608
Precision: 0.06770274909170233
Recall (Sensitivity): 0.5805154639175257
F1-Score: 0.12109267758412584
------------------------------------------------------------------------
General results:
MLP R-value: 0.0644452121077323
ANFIS R-value: 0.0989188247848439
Average MLP portfolio appreciation: 5.751790000000001
Average ANFIS portfolio appreciation: 7.011390000000001
Average Market portfolio appreciation: 8.565


In [200]:
np.average(mlp_rvalues_dict['MLP R-Value'])

0.6787113140537798